In [1]:
import numpy as np
import pandas as pd

 - **Layers**: an Integer value representing the total number of hidden layers in the network (input and output layers are extra).
 
 - **Nodes**: an integer array of size [0,..,Layers+1] containing the dimensions of the neural
network. Nodes[0] shall represent the input size (typically, 50), Nodes[Layers+1]
shall represent the number of output nodes (typically, 1). All other values Nodes[i]
represent the number of nodes in hidden layer i.

 - **NNodes**: a possible alternative to the Nodes parameter for situations where you want
each hidden layer of the neural network to be of the same size. In this case, the size of
the output layer is assumed to be 1, and the size of the input layer can be inferred from
the dataset.

 - **Activations**: an array of size [0,..,Layers+1] (for the sake of compatibility) in which
Activations[0] and Activations[Layers+1] are not used, while all other
Activations[i] values are labels indicating the activation function used in layer i.
This allows you to build neural networks with different activation functions in each layer.

 - **ActivationFn**: a possible alternative to Activations when all hidden layers of your neural
network use the same activation function.

In [2]:
class NeuralNetwork:
    #Layers: an Integer value representing the total number of hidden layers in the network 
    #        (input and output layers are extra)

    def __init__(self, Layers, Nodes, NNodes, Activations, ActivationFn):
        self.Layers = Layers
        self.Nodes = Nodes
        self.NNodes = NNodes
        self.Activations = Activations
        self.ActivationFn = ActivationFn
        
    # Forward Pass
        
    def Relu(self,e):
        return max(0,e)
    
    def leakyRelu(self,e,a=0.1):
        if e > 0:
            return e
        else:
            return a*e
        
    def sigmoid(self,e):
        return 1/(1+np.exp(1)**-e)
    
    def tanh(self,e):
        return 2*sigmoid(2*e) - 1
    
    def applyActivation(self,layer,i):
        acttype = self.Activations[i]
        if acttype == "Relu":
            return layer.applymap(self.Relu)
        elif acttype == "leaky":
            return layer.applymap(self.leakyRelu)
        elif acttype == "sigmoid":
            return layer.applymap(self.sigmoid)
        elif acttype == "tanh":
            return layer.applymap(self.tanh)
    
    def loss(self,z,y):
        # Performs L2 loss (for this project)
        L = 0.5*(np.array(z)-np.array(y))**2 # Assumes the squaring is element wise
        L = np.sum(L) * (1/len(z)) # Take average of all the losses
        return L
    
    
        
        
    def forward_pass(self, X, y, weights):
        # Assume X already has a column of ones for bias term.
        # Assume weights include the weights for the bias term when going into next layer
        
        savings = [X]
        
        # From input layer to first hidden layer
        h = X.dot(weights[0]) # Get first hidden layer without the bias node added in
        h['ones'] = 1 # Add in bias node to the hidden layer
        savings.append(h) # Saving intermediate values
        hact = self.applyActivation(h,0) # Perform activation
        hact['ones'] = 1
        savings.append(hact) # Saving intermediate values
        h = hact
     
        for i in range(1,len(weights)):
            if i != len(weights)-1: # A hidden layer
                h = h.dot(weights[i])
                h['ones'] = 1 # Add in bias node to the hidden layer
                savings.append(h) # Saving intermediate values
                hact = self.applyActivation(h,i) # Perform activation
                hact['ones'] = 1
                savings.append(hact) # Saving intermediate values
                h = hact
            else: # For Z value/vector
                z = h.dot(weights[i])
                savings.append(z)
                
                # Calculate loss
                L = self.loss(z,y)
                savings.append(L) # Are we saving average loss over the batch?
                
        return savings
    
    # Backwards pass
        def J_loss(self,z,y):
            B = len(y)
            J = (1/B)*(np.array(z) - np.array(y))
            return J
    
    def back_propagation(self,X,y,intermediates,weights, lr):
        J = self.J_loss(intermediates[-2],y) # Compute the jacobian of the loss layer evaluated at z
        w_on = True
        w_count = len(weights)-1
        for i in range(-3,-len(intermediates),-1):
            if w_on:
                J_wn = J.dot(self.J_weight(intermediates[i])) # Calculate the jacobian of the weights evaluated at sigma
                weights[w_count] = weights[w_count] - lr*J_wn # Update the weights
                J = J.dot(self.J_inputlayer(intermediates[i])) # Update jacobian by computing the jacobian of dense layer wrt input
                w_count = w_count - 1 # Update the index for the next set of weights
                w_on = False # Next derivative evaluated at intermediates[i] will not update the weights
            else:
                J = J.dot(self.J_sigma(intermediates[i]))
                w_on = True
        # Update last set of weights (W_1)
        J_w1 = J.dot(self.J_weight(intermediates[-len(intermediates)]))
        weights[w_count] = weights[w_count] - lr*J_w1
        return weights

    
    # Training
    
    def initialize_weights(self):
        weights = []
        for i in range(len(self.Nodes)-1):
            M = self.Nodes[i] + 1  #+1 for bias term
            N = self.Nodes[i+1]  
            w = np.random.normal(loc=0,scale = np.sqrt(2/(M+N)),size=(M,N)) 
            weights.append(w)
        return weights
    
    def train(self,X,y,lr,batch):
        pass

In [8]:
df = pd.read_csv('datasets/sberbank-russian-housing-market/train.csv')
df = df.select_dtypes(exclude=['category', 'object'])
df = df.drop(['id'], axis=1)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,-0.294873,-0.141596,-0.690013,NaN,NaN,NaN,NaN,NaN,NaN,-0.544788,...,-0.313871,-0.238993,-0.326975,-0.070250,-0.174277,0.915176,-0.420245,-0.017208,-0.406425,-0.266324
1,-0.531523,-0.294604,-0.877987,NaN,NaN,NaN,NaN,NaN,NaN,-0.390702,...,-0.232198,-0.274222,-0.326975,-0.001564,-0.026433,0.915176,0.065654,0.285221,1.638925,-0.234943
2,-0.294873,-0.103343,-1.065960,NaN,NaN,NaN,NaN,NaN,NaN,-0.622239,...,-0.300259,-0.274222,-0.326975,-0.138936,-0.068674,-0.726162,-0.225885,0.306823,0.820785,-0.297704
3,0.914671,0.298304,0.249854,NaN,NaN,NaN,NaN,NaN,NaN,-0.245700,...,-0.286647,-0.309452,-0.142429,-0.379335,-0.554448,-0.726162,-0.420245,-0.578861,-0.610960,1.250402
4,0.599138,0.814708,-0.690013,NaN,NaN,NaN,NaN,NaN,NaN,-0.448374,...,3.905860,3.424887,2.810299,4.119571,4.345529,2.556515,4.001437,3.071885,1.638925,1.926434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,-0.268578,-0.141596,-0.126093,-0.526757,-0.558443,-0.007080,0.105890,-0.014127,1.014604,-0.368245,...,-0.232198,-0.203763,-0.326975,-0.001564,-0.089795,0.915176,-0.323065,0.674058,0.002645,0.057942
30467,0.835788,0.470439,-0.877987,-0.526757,0.116722,-0.007339,2.453903,0.127389,1.014604,-0.501211,...,3.824188,4.129479,4.102117,2.848887,3.205018,0.915176,3.564128,2.553436,1.843460,3.739925
30468,-0.242284,NaN,0.437827,1.101332,-0.558443,NaN,-1.068116,-0.191022,-1.257808,0.381575,...,-0.422767,-0.344682,-0.326975,-0.448021,-0.385483,-0.726162,-0.371655,-0.902892,-1.020030,-0.031815
30469,0.257310,-0.045965,-0.502040,0.361292,-0.558443,-0.006899,0.105890,0.162768,-0.121602,-0.562102,...,-0.136914,-0.344682,-0.142429,-0.310650,0.015808,0.915176,-0.225885,0.263619,0.207180,1.334084


In [10]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

In [11]:
df

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,-0.294873,-0.141596,-0.690013,-0.378749,-0.558443,-0.007087,0.692894,0.021252,1.014604,-0.544788,...,-0.313871,-0.238993,-0.326975,-0.070250,-0.174277,0.915176,-0.420245,-0.017208,-0.406425,-0.266324
1,-0.531523,-0.294604,-0.877987,-0.008729,2.142218,-0.007090,-1.068116,0.074321,0.446501,-0.390702,...,-0.232198,-0.274222,-0.326975,-0.001564,-0.026433,0.915176,0.065654,0.285221,1.638925,-0.234943
2,-0.294873,-0.103343,-1.065960,-0.822773,0.116722,-0.007142,1.279897,-0.067196,-0.121602,-0.622239,...,-0.300259,-0.274222,-0.326975,-0.138936,-0.068674,-0.726162,-0.225885,0.306823,0.820785,-0.297704
3,0.914671,0.298304,0.249854,0.657308,-0.558443,-0.006931,1.279897,0.162768,1.014604,-0.245700,...,-0.286647,-0.309452,-0.142429,-0.379335,-0.554448,-0.726162,-0.420245,-0.578861,-0.610960,1.250402
4,0.599138,0.814708,-0.690013,-0.896777,0.116722,-0.007258,1.866900,0.021252,0.446501,-0.448374,...,3.905860,3.424887,2.810299,4.119571,4.345529,2.556515,4.001437,3.071885,1.638925,1.926434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,-0.268578,-0.141596,-0.126093,-0.526757,-0.558443,-0.007080,0.105890,-0.014127,1.014604,-0.368245,...,-0.232198,-0.203763,-0.326975,-0.001564,-0.089795,0.915176,-0.323065,0.674058,0.002645,0.057942
30467,0.835788,0.470439,-0.877987,-0.526757,0.116722,-0.007339,2.453903,0.127389,1.014604,-0.501211,...,3.824188,4.129479,4.102117,2.848887,3.205018,0.915176,3.564128,2.553436,1.843460,3.739925
30468,-0.242284,0.135733,0.437827,1.101332,-0.558443,-0.019870,-1.068116,-0.191022,-1.257808,0.381575,...,-0.422767,-0.344682,-0.326975,-0.448021,-0.385483,-0.726162,-0.371655,-0.902892,-1.020030,-0.031815
30469,0.257310,-0.045965,-0.502040,0.361292,-0.558443,-0.006899,0.105890,0.162768,-0.121602,-0.562102,...,-0.136914,-0.344682,-0.142429,-0.310650,0.015808,0.915176,-0.225885,0.263619,0.207180,1.334084


In [ ]:
df_corr = df.corr()
df_corr

In [ ]:
remove_cols = set()
for col1 in df.columns:
    if col1 in remove_cols:
        continue
        
    for col2 in df.columns:
        if col1 == col2 or col2 in remove_cols:
            continue
            
        if abs(df_corr[col1][col2]) > 0.80:
            remove_cols.add(col2)

In [ ]:
len(remove_cols)

In [ ]:
df.drop(list(remove_cols), axis=1)

In [ ]:
df[['timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq']]

In [ ]:
X = iris[["sepal_length", "sepal_width", "petal_length"]]
X['ones'] = 1
y = iris[["petal_width"]]

In [ ]:
NN = NeuralNetwork(Layers=1, Nodes=[3,2,1], NNodes=None, Activations=["Relu","Relu"], ActivationFn=None)
weights = NN.initialize_weights()
intermediates = NN.forward_pass(X,y,weights)

In [ ]:
intermediates[4]

In [ ]:
B=130
(1/B)*(np.array(intermediates[3]) - np.array(y))

# -------------------------------------------------------------------------------------------

In [ ]:
def ReLu(e):
    return max(0,e)

In [ ]:
def leakyRelu(e,a=0.1):
    if e > 0:
        return e
    else:
        return a*e

In [ ]:
def sigmoid(e):
    return 1/(1+np.exp(1)**-e)

In [ ]:
def tanh(e):
    return 2*sigmoid(2*e) - 1

In [ ]:
df = iris.drop('species',axis=1)
# b = petal_width

In [ ]:
w = np.random.normal(loc=0,scale = np.sqrt(2/(3+3)),size=(3,4))
w

w = np.array(df.mean())
w = [list(w)]*3
w = np.array(w)
w

In [ ]:
one = np.array(df.iloc[0])
one[len(one)-1] = 1
one

In [ ]:
h = w.dot(one)
h

In [ ]:
pd.Series(h).apply(tanh)

In [ ]:
h = pd.Series(h).apply(leakyRelu,args=(0.2,))
h

In [ ]:
b2 = 2
w2 = np.array([1,2,3,b2])
z = w2.dot(np.append(h,1))

In [ ]:
z

In [ ]:
df["petal_width"] = 1

In [ ]:
df

In [ ]:
w

In [ ]:
w.T

In [ ]:
h = df.dot(w.T).applymap(ReLu)
#df.dot(w.T)
h

In [ ]:
h["ones"] = 1

In [ ]:
h

In [ ]:
z = h.dot(w2)

In [ ]:
# Average Loss over batch
Lb = 0.5*((z-iris["petal_width"])**2)
(1/len(Lb))*np.sum(Lb)